In [1]:
%load_ext autoreload
%autoreload 2
%cd '/home/xingyu/Projects/PlasticineLab'
%pwd

import torch.multiprocessing as mp

try:
    mp.set_start_method('spawn')
except RuntimeError:
    pass

import json
from chester import logger
import numpy as np
import torch
import argparse
import os
from rl_td3.td3 import TD3
from imitation.buffer import ReplayBuffer
from imitation.env_spec import get_tool_spec
from imitation.eval_helper import get_eval_traj
from plb.utils.visualization_utils import make_grid, save_numpy_as_gif
from plb.envs.mp_wrapper import make_mp_envs
from rl_td3.profiler import Profiler
import tqdm
from imitation.train_full import get_args
exp_folder = 'data/hza/rl/gathermove-0-100/0928_td3/0928_td3_2021_10_01_06_11_21_0001/'

with open(os.path.join(exp_folder, 'variant.json'), 'rb') as f:
    vv = json.load(f)
args = get_args()
args.__dict__.update(**vv)
args.num_env = 1
# Configure logger
logger.configure(dir='./data/eval_rl', exp_name='')
log_dir = logger.get_dir()
assert log_dir is not None
os.makedirs(log_dir, exist_ok=True)
print("number of devices in curretn env ", torch.cuda.device_count())

# ----------preparation done------------------
env = make_mp_envs(args.env_name, args.num_env, args.seed)
args.cached_state_path = env.getattr('cfg.cached_state_path', 0)

spec = get_tool_spec(env, args.env_name)
args.action_mask = spec['action_masks'][args.tool_combo_id]
args.discount = float(args.gamma)

obs_channel = len(args.img_mode) * args.frame_stack
obs_shape = (args.image_dim, args.image_dim, obs_channel)
action_dim = env.getattr('action_space.shape[0]', 0)
max_action = float(env.getattr('env.action_space.high[0]', 0))

kwargs = {
    'args': args,
    "obs_shape": obs_shape,
    "action_dim": action_dim,
    "max_action": max_action,
    "discount": args.discount,
    "tau": args.tau}

# Initialize policy
if args.policy == "TD3":
    # Target policy smoothing is scaled wrt the action scale
    kwargs["policy_noise"] = args.policy_noise * max_action
    kwargs["noise_clip"] = args.noise_clip * max_action
    kwargs["policy_freq"] = args.policy_freq
    policy = TD3(**kwargs)
else:
    raise NotImplementedError

/home/xingyu/Projects/PlasticineLab
Logging to ./data/eval_rl
number of devices in curretn env  2


In [15]:
import glob
filename = glob.glob(os.path.join(exp_folder, 'model_*_actor.pth'))[0]
filename = filename[:-10]
print(filename)
policy.load(filename)

tag = args.env_name + '_' + str(args.tool_combo_id) + '_' + str(args.seed)
print(tag)
from rl.eval_td3 import eval_policy
eval_policy(args, policy, env, args.seed, tag)

data/hza/rl/gathermove-0-100/0928_td3/0928_td3_2021_10_01_06_11_21_0001/model_47500
GatherMove-v1_0_100
MoviePy - Building file ./data/eval_rl/eval_GatherMove-v1_0_100.gif with imageio.


successes: [0, 0, 0, 0, 0]
